***
## Локализация и перевод

В Django есть встроенная система интернационализации, она поддерживает множество языков. 

Чтобы переключить интерфейс админки на другой язык, надо [найти кодовое обозначение нужного языка](http://www.i18nguy.com/unicode/language-identifiers.html) и подставить его в константу LANGUAGE_CODE в файле settings.py:

```py
# anfisa_for_friends/settings.py

# Переводим на русский!
LANGUAGE_CODE = 'ru-RU' 
```

При изменении значения константы `LANGUAGE_CODE` на `'ru-RU'` все тексты встроенных веб-интерфейсов Django будут отображаться по-русски; помимо этого, проект будет и локализован: даты будут отображаться в привычном нам формате.

***
## Verbose name: переводим названия приложения и моделей

Название приложений придуманы разработчиком, и перевести эти названия должен сам разработчик: в файле интернационализации таких переводов точно нет. 

Задать название приложения в том виде, как оно должно отображаться в интерфейсах Django, можно в файле ***apps.py***.

Добавьте в класс `IceCreamConfig` свойство `verbose_name` со значением `'Каталог мороженого'`; сохраните файл, обновите страницу админ-зоны: название раздела в админке изменится. 

```py
# ice_cream/apps.py
from django.apps import AppConfig

class IceCreamConfig(AppConfig):
    default_auto_field = 'django.db.models.BigAutoField'
    name = 'ice_cream'
    # Добавить строчку:
    verbose_name = 'Каталог мороженого'
```

Выражение `verbose_name` в примерном переводе означает «расширенное название».

***
## Переводим название модели

Управление записями БД в админке разбито на разделы в соответствии с моделями; разделы названы по названиям моделей.

Но администратор сайта, скорее всего, понятия не имеет о моделях: для него слова *Categorys*, *Ice creams*, *Topping* и *Wrapper* — просто заголовки на странице. И он удивится: «почему было не назвать разделы понятно?». Не проблема, поможем админу.

Название модели, которое будет отображаться в админке, нужно указать в классе `Meta` этой модели. 

Понадобится добавить два свойства:

* `verbose_name` — расширенное название,

* `verbose_name_plural` — расширенное название во множественном числе.

```py
# ice_cream/models.py
...

class Category(PublishedModel):
    title = models.CharField(max_length=256)
    slug = models.SlugField(max_length=64, unique=True)
    output_order = models.PositiveSmallIntegerField(default=100)

    class Meta:
        verbose_name = 'категория'
        verbose_name_plural = 'Категории' 
```

***
## Переводим названия атрибутов моделей

Названия атрибутов моделей отображаются, например, на страницах создания или редактирования объекта в админ-зоне.

В админ-зоне заголовки полей ввода генерируются из названий атрибутов модели и выглядят они непонятно; лучше дать этим полям нормальные читаемые заголовки. В Django предусмотрена такая возможность: для каждого из атрибутов модели можно указать аргумент `verbose_name`:

```py
# ice_cream/models.py
...

class Category(PublishedModel):
    title = models.CharField(max_length=256, verbose_name='Название')
    slug = models.SlugField(max_length=64, unique=True, verbose_name='Слаг')
    output_order = models.PositiveSmallIntegerField(
        default=100,
        verbose_name='Порядок отображения'
    ) 
```

Человекочитаемое название поля можно указать первым позиционным аргументом:

```py
# ice_cream/models.py
...

class Category(PublishedModel):
    title = models.CharField('Название', max_length=256)
    slug = models.SlugField('Слаг', max_length=64, unique=True)
    output_order = models.PositiveSmallIntegerField(
        'Порядок отображения',
        default=100
    )
```

Эти варианты взаимозаменяемы, однако последний вариант не всегда применим. Например, при указании типа поля ForeignKey первым параметром указывается связанная модель.

При наследовании от абстрактных моделей `verbose_name` поля надо указывать в абстрактной модели. В проекте **anfisa_for_friends** поле `is_published` вынесено в абстрактную модель `PublishedModel`, так что переводить его нужно в описании этой модели.

В связанных моделях также можно использовать `verbose_name`. Посмотрите, что будет, если сделать:

```py
# ice_cream/models.py

...

class IceCream(PublishedModel):
    ...
    category = models.ForeignKey(
        Category,
        on_delete=models.CASCADE,
        related_name='ice_creams',
        verbose_name='Категория'
    )
... 
```

***
## Выводим читаемые названия объектов

Список объектов любой из моделей выглядит в админке не очень презентабельно и совершенно непригоден для управления данными: как понять, что за сорт мороженого кроется, например, под заголовком *IceCream object (8)*?

Настроим вывод записей так, чтобы в качестве заголовка показывалось название мороженого (значение поля `title`). Те же настройки подойдут и для других моделей.

Для этого в модели нужно переопределить «магический» метод `__str__()` и указать в нём, какое поле должно возвращаться в качестве строкового представления объекта:

```py
# ice_cream/models.py
...

class Category(PublishedModel):
    title = models.CharField(max_length=256, verbose_name='Название')
    slug = models.SlugField(max_length=64, unique=True, verbose_name='Слаг')
    output_order = models.PositiveSmallIntegerField(
        default=100,
        verbose_name='Порядок отображения'
    )

    class Meta:
        verbose_name = 'категория'
        verbose_name_plural = 'Категории'

    def __str__(self):
        return self.title
```

По [правилам оформления кода Django](https://docs.djangoproject.com/en/dev/internals/contributing/writing-code/coding-style/#model-style) метод `__str__()` полагается определять после класса `Meta`.